### Particle Filter Boilerplate Code

In [ ]:
import numpy as np

# --- CONFIGURATION ---
N = 1000             # Number of particles
STATE_DIM = 3        # e.g., [x, y, theta]
NOISE_COV = np.diag([0.1, 0.1, 0.05]) # The covariance matrix (R) for adding noise

# 1. THE MOTION MODEL (Line 4)
# Input: Current particle states (N x D), Control input u
# Output: Predicted particle states (Deterministic)
def motion_model(particles, u):
    # TODO: Fill this in with your physics equations.
    # Example (simple 2D movement):
    # particles[:, 0] += u[0] # Move x
    # particles[:, 1] += u[1] # Move y
    return particles

# 2. THE MEASUREMENT MODEL (Line 5)
# Input: Particle states (N x D), Actual Sensor Reading z
# Output: Unnormalized weights (N,) - Probability 0.0 to 1.0
def measurement_model(particles, z):
    # TODO: Fill this in with your sensor logic (Gaussian Likelihood).
    # This function asks: "How likely is observation 'z' if the robot is at 'particle'?"
    
    # Example (Placeholder):
    # dist = np.linalg.norm(particles[:, :2] - landmark, axis=1)
    # weights = np.exp(-0.5 * (dist - z)**2 / sensor_noise)
    
    # For now, return uniform weights so code runs
    return np.ones(len(particles))

In [ ]:
# --- INITIALIZATION ---
# Create the initial cloud (Line 2)
# Initialize N particles randomly within some bounds
particles = np.random.uniform(low=-10, high=10, size=(N, STATE_DIM))

# Initialize weights uniformly (1/N)
weights = np.ones(N) / N

# --- MAIN LOOP ---
# This simulates the loop "for m = 1 to M" over time steps
for t in range(100):
    
    # 1. GET DATA (In a real scenario, this comes from the robot)
    u_t = [1.0, 0.5]  # Control command
    z_t = 5.2         # Sensor reading
    
    # ------------------------------------------------------
    # STEP A: PREDICT / SAMPLING (Line 4)
    # ------------------------------------------------------
    
    # A.1: Apply the Deterministic Motion Model (The "x_bar")
    particles = motion_model(particles, u_t)
    
    # A.2: Add Random Noise (Sample from Covariance Matrix)
    # This creates the distribution spread
    noise = np.random.multivariate_normal(
        mean=np.zeros(STATE_DIM), 
        cov=NOISE_COV, 
        size=N
    )
    particles += noise

    # ------------------------------------------------------
    # STEP B: UPDATE / SCORING (Line 5)
    # ------------------------------------------------------
    
    # Calculate how well each particle explains the sensor data
    # "weights" here corresponds to w_t in the diagram
    weights = measurement_model(particles, z_t)
    
    # Normalize weights so they sum to 1.0 (Crucial for probability)
    weights += 1.e-300          # Avoid division by zero
    weights /= np.sum(weights)
    
    # ------------------------------------------------------
    # STEP C: RESAMPLE (Lines 8-11)
    # ------------------------------------------------------
    
    # 1. The Effective N check (Optional but recommended)
    # If weights are very skewed, we resample. If distinct enough, we might skip.
    # For this simple implementation, we resample every time.
    
    # 2. Draw indices based on weight probability
    indices = np.random.choice(N, size=N, p=weights, replace=True)
    
    # 3. Create the new set (Line 10)
    # This automatically "clones" good particles and "kills" bad ones
    particles = particles[indices]
    
    # 4. Reset weights for the next round
    weights = np.ones(N) / N

    # --- ESTIMATION (Optional) ---
    # Where do we think we are?
    estimated_position = np.mean(particles, axis=0)
    # print(f"Step {t}: {estimated_position}")